In [1]:
from rich.pretty import install
from rich import inspect
install()
def what(obj, *args, **kwargs):
    return inspect(obj, *args, dunder=kwargs.pop('dunder', True), **kwargs)
def uniquedir(obj):
    return sorted(set(dir(obj)) - set(dir(object)) - set(dir(type)))
def is_from_ipython(prop):
    return (prop.startswith("_i") or (len(prop) > 1 and prop[1].isdigit()) or prop in ("In", "Out", "_", "__", "___", "_dh", "_oh", "get_ipython", "exit", "open", "quit", "__builtin__",))
from typing import TypeVar, Generic, Type
TInstance = TypeVar("TInstance")
TProperty = TypeVar("TProperty", bound=Type)


# Now let's have some fun. Welcome to dunderfest! 🥳

In [12]:
class Team:
    members: list[Teammate] = []

    def __init__(self, *team_members) -> None:
        self.members = list(map(Teammate, team_members))

    def __iter__(self):
        return iter(self.members)

    def start_daily(self):
        print(f'Starting daily!')
        for member in self:
            print(member, end=' ')
            member.tell_everyone_what_im_working_on()


In [11]:
class Teammate:
    name: str = StronglyTypedProperty()
    tasks: Dictionary[str, Task]
    tired = False

    def __init__(self, name: str):
        self.name = name

    @cached_property
    def tasks(self):
        tasks_regular_dict = get_tasks(self.name)
        cast_tasks = {task_name: Task(task) for task_name, task in tasks_regular_dict.items()}
        cool_dictionary_of_tasks = Dictionary(cast_tasks)
        return cool_dictionary_of_tasks
    
    def tell_everyone_what_im_working_on(self):
        print(f"Working on {self.tasks}\n")

In [10]:
class Task(Dictionary, metaclass=StronglyTypedMetaClass):
    id: str
    status: str
    started: str
    story_points: int
    priority: str

    def __hash__(self):
        return hash(self.id)

    def __eq__(self, other):
        """Task(1) == Task(2)"""
        return self.id == other.id

    def __gt__(self, other):
        return self.priority == "critical" and other.priority == "high"


In [9]:
class StronglyTypedProperty(Generic[TInstance, TProperty]):
    """Yells at you if you set the wrong type"""
    
    def __init__(self, name: str = None, type: TProperty = None):
        self.name = name
        self.type = type

    def __get__(self, instance: TInstance, cls: Type[TInstance]):
        return instance.__dict__[self.name]

    def __set__(self, instance: TInstance, value: TProperty):
        if not isinstance(value, self.type):
            raise TypeError(f"Property '{self.name}' must be of type {self.type} (got {type(value)})")
        instance.__dict__[self.name] = value

    def __delete__(self, instance: TInstance):
        raise TypeError(f"{self.name} cannot be deleted, it is strong! 💪")

    def __set_name__(self, cls: Type[TInstance], property_name: str):
        self.name = property_name
        self.type = cls.__annotations__[property_name]


In [8]:
class Dictionary(dict):
    """
    this.is.better
    this['is']['annoying']
    """
    
    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(key)

    def __setattr__(self, key, value):
        self[key] = value

    def __delattr__(self, key):
        try:
            del self[key]
        except KeyError:
            raise AttributeError(key)

    __dict__ = property(lambda self: self)


In [7]:
class StronglyTypedMetaClass(type):
    """Automatically makes your `field: type` a StronglyTypedProperty"""

    def __init__(cls, cls_name: str, bases: tuple, namespace: dict):
        for property_name, type_annotation in cls.__annotations__.items():
            existing_field = getattr(cls, property_name, None)
            if not isinstance(existing_field, StronglyTypedProperty):
                strongly_typed_property = StronglyTypedProperty(property_name, type_annotation)
                setattr(cls, property_name, strongly_typed_property)
        super().__init__(cls_name, bases, namespace)



In [6]:
class cached_property:
    def __init__(self, function):
        self.function = function

    def __get__(self, instance, cls):
        if instance is None:
            return self
        value = self.function(instance)
        instance.__dict__[self.function.__name__] = value  # Removes its own reference! 🤯
        return value


In [5]:
def get_tasks(url: str) -> dict:
    return {
        "cache_invalidation": {
            "id":           "000",
            "status":       "in_review",
            "started":      "long time ago",
            "story_points": 1,
            "priority":     "high",
            },
        "raising_ami":        {
            "id":           "001",
            "status":       "in_progress",
            "started":      "10 months ago",
            "story_points": 999999,
            "priority":     "critical",
            },
        }

In [16]:
bob = Team("Einstein", "Newton", "Tesla")

In [14]:
bob.start_daily()

Starting daily!
<__main__.Teammate object at 0x107401e70> Working on {'cache_invalidation': {'id': '000', 'status': 'in_review', 'started': 'long time ago', 'story_points': 1, 'priority': 'high'}, 'raising_ami': {'id': '001', 'status': 'in_progress', 'started': '10 months ago', 'story_points': 999999, 'priority': 'critical'}}

<__main__.Teammate object at 0x107401060> Working on {'cache_invalidation': {'id': '000', 'status': 'in_review', 'started': 'long time ago', 'story_points': 1, 'priority': 'high'}, 'raising_ami': {'id': '001', 'status': 'in_progress', 'started': '10 months ago', 'story_points': 999999, 'priority': 'critical'}}

<__main__.Teammate object at 0x107401870> Working on {'cache_invalidation': {'id': '000', 'status': 'in_review', 'started': 'long time ago', 'story_points': 1, 'priority': 'high'}, 'raising_ami': {'id': '001', 'status': 'in_progress', 'started': '10 months ago', 'story_points': 999999, 'priority': 'critical'}}



### Let's make it more readable

In [15]:
class Teammate:
    name: str = StronglyTypedProperty()
    tasks: Dictionary[str, Task]
    tired = False

    def __init__(self, name: str):
        self.name = name

    @cached_property
    def tasks(self):
        tasks_regular_dict = get_tasks(self.name)
        cast_tasks = {task_name: Task(task) for task_name, task in tasks_regular_dict.items()}
        cool_dictionary_of_tasks = Dictionary(cast_tasks)
        return cool_dictionary_of_tasks

    def tell_everyone_what_im_working_on(self):
        print(f"Working on {self.tasks}\n")
    
    # New:
    def __str__(self):
        return f"{self.name} has {len(self.tasks)} tasks. He is {'tired' if self.tired else 'not tired'}."

    def __repr__(self):
        return f"{self.__class__.__name__}('{self.name}')"

In [17]:
bob.start_daily()

Starting daily!
Einstein has 2 tasks. He is not tired. Working on {'cache_invalidation': {'id': '000', 'status': 'in_review', 'started': 'long time ago', 'story_points': 1, 'priority': 'high'}, 'raising_ami': {'id': '001', 'status': 'in_progress', 'started': '10 months ago', 'story_points': 999999, 'priority': 'critical'}}

Newton has 2 tasks. He is not tired. Working on {'cache_invalidation': {'id': '000', 'status': 'in_review', 'started': 'long time ago', 'story_points': 1, 'priority': 'high'}, 'raising_ami': {'id': '001', 'status': 'in_progress', 'started': '10 months ago', 'story_points': 999999, 'priority': 'critical'}}

Tesla has 2 tasks. He is not tired. Working on {'cache_invalidation': {'id': '000', 'status': 'in_review', 'started': 'long time ago', 'story_points': 1, 'priority': 'high'}, 'raising_ami': {'id': '001', 'status': 'in_progress', 'started': '10 months ago', 'story_points': 999999, 'priority': 'critical'}}



In [18]:
class Team:
    members: list[Teammate] = []

    def __init__(self, *team_members) -> None:
        self.members = list(map(Teammate, team_members))

    def __iter__(self):
        return iter(self.members)

    def start_daily(self):
        print(f'Starting daily!')
        for member in self:  # __iter__
            print(member, end=' ')
            member.tell_everyone_what_im_working_on()

    # New:
    def __contains__(self, item):
        return item in self.members

    def __getitem__(self, slice):
        return self.members[slice]
    
bob = Team("Einstein", "Newton", "Tesla")

In [19]:
einstein = bob[0]  # __getitem__

In [20]:
einstein           # __repr__

Teammate('Einstein')

In [21]:
einstein in bob    # __contains__

True

In [23]:
einstein.name = None  # StronglyTypedProperty

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/6d/v_8150015qz_fxg7lkd8qgxw0000gn/T/ipykernel_95875/2649339167.py:1 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/6d/v_8150015qz_fxg7lkd8qgxw0000gn/T/ipykernel_95875/2649339167.py'                 │
│                                                                                                  │
│ /var/folders/6d/v_8150015qz_fxg7lkd8qgxw0000gn/T/ipykernel_95875/503597796.py:13 in __set__      │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/6d/v_8150015qz_fxg7lkd8qgxw0000gn/T/ipykernel_95875/503597796.py'                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: Property 'name' must be of type <class 'str'> (got <class 'NoneType'>)

In [25]:
class Teammate:
    name: str = StronglyTypedProperty()
    tasks: Dictionary[str, Task]
    tired = False

    def __init__(self, name: str):
        self.name = name

    @cached_property
    def tasks(self):
        tasks_regular_dict = get_tasks(self.name)
        cast_tasks = {task_name: Task(task) for task_name, task in tasks_regular_dict.items()}
        cool_dictionary_of_tasks = Dictionary(cast_tasks)
        return cool_dictionary_of_tasks

    def tell_everyone_what_im_working_on(self):
        print(f"Working on {self.tasks}\n")
    
    def __str__(self):
        return f"{self.name} has {len(self.tasks)} tasks. He is {'tired' if self.tired else 'not tired'}."

    def __repr__(self):
        return f"{self.__class__.__name__}('{self.name}')"
    
    # New:
    def __enter__(self):
        if self.tired:
            raise BrokenPipeError("I need to sleep")
        return self

    def __exit__(self, exception_type, exception_instance, traceback):
        self.tired = True  # regardless. lol

    def __call__(self, tasks: dict):
        self.tasks.update(tasks)

bob = Team("Einstein", "Newton", "Tesla")
einstein = bob[0]

In [26]:
with einstein as yalla_einstein:  # __enter__
    yalla_einstein({  # __call__
        "general_relativity":
            Task(id="003", status="to_do", started="1905", story_points=3, priority="high")
        })
# __exit__

In [27]:
str(einstein)

'Einstein has 3 tasks. He is tired.'

In [28]:
with einstein as yalla_einstein:  # __enter__
    yalla_einstein({
        "quantum_mechanics":
            Task(id="004", status="to_do", started="1915", story_points=4, priority="high")
        })

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/6d/v_8150015qz_fxg7lkd8qgxw0000gn/T/ipykernel_95875/3581169787.py:1 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/6d/v_8150015qz_fxg7lkd8qgxw0000gn/T/ipykernel_95875/3581169787.py'                 │
│                                                                                                  │
│ /var/folders/6d/v_8150015qz_fxg7lkd8qgxw0000gn/T/ipykernel_95875/965098828.py:28 in __enter__    │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/6d/v_8150015qz_fxg7lkd8qgxw0000gn/T/ipykernel_95875/965098828.py'                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
BrokenPipeError: I need to sleep

In [30]:
class Teammate:
    name: str = StronglyTypedProperty()
    tasks: Dictionary[str, Task]
    tired = False

    def __init__(self, name: str):
        self.name = name

    @cached_property
    def tasks(self):
        tasks_regular_dict = get_tasks(self.name)
        cast_tasks = {task_name: Task(task) for task_name, task in tasks_regular_dict.items()}
        cool_dictionary_of_tasks = Dictionary(cast_tasks)
        return cool_dictionary_of_tasks

    def tell_everyone_what_im_working_on(self):
        print(f"Working on {self.tasks}\n")
    
    def __str__(self):
        return f"{self.name} has {len(self.tasks)} tasks. He is {'tired' if self.tired else 'not tired'}."

    def __repr__(self):
        return f"{self.__class__.__name__}('{self.name}')"
    
    def __enter__(self):
        if self.tired:
            raise BrokenPipeError("I need to sleep")
        return self

    def __exit__(self, exception_type, exception_instance, traceback):
        self.tired = True  # regardless. lol

    def __call__(self, tasks: dict):
        self.tasks.update(tasks)
        
    def __bool__(self):
        return not self.tired


bob = Team("Einstein", "Newton", "Tesla")
einstein = bob[0]
with einstein:
    pass

In [31]:
def start_daily(self):
    print(f'Starting daily!')
    for member in self:
        print(member)
        if member:  # __bool__
            member.tell_everyone_what_im_working_on()

In [32]:
start_daily(bob)

Starting daily!
Einstein has 2 tasks. He is tired.
Newton has 2 tasks. He is not tired.
Working on {'cache_invalidation': {'id': '000', 'status': 'in_review', 'started': 'long time ago', 'story_points': 1, 'priority': 'high'}, 'raising_ami': {'id': '001', 'status': 'in_progress', 'started': '10 months ago', 'story_points': 999999, 'priority': 'critical'}}

Tesla has 2 tasks. He is not tired.
Working on {'cache_invalidation': {'id': '000', 'status': 'in_review', 'started': 'long time ago', 'story_points': 1, 'priority': 'high'}, 'raising_ami': {'id': '001', 'status': 'in_progress', 'started': '10 months ago', 'story_points': 999999, 'priority': 'critical'}}



In [33]:
from pathlib import Path